In [54]:
# FOLDER = '../data/olivia/Healthy'
FOLDER = '../data/olivia/Flat feet'
EXTENSION = '.obj'
# EXTENSION = '.stl'
EXAMPLE_FILE = '../data/stl/020_l.stl'
EXAMPLE_LMRK = '../data/stl/020_l.pkl'
EXAMPLE_EXTRA_LMRK = '../data/stl/020_l-extra.pkl'
# OUTPUT_FOLDER = 'output/olivia/Healthy'
OUTPUT_FOLDER = 'output/olivia/Flat feet'

In [55]:
import sys
sys.path.append('..')

## Load data

In [56]:
import os

files = os.listdir(FOLDER)
files = [os.path.join(FOLDER, f) for f in files if EXTENSION in f]
files.sort()
files

['../data/olivia/Flat feet/AM002R.obj']

In [57]:
# load landmarks labelling
import pandas as pd

lmrk_dict = {}

for file in files:
    lmrk = pd.read_pickle(file.replace(EXTENSION, '.pkl'))
    lmrk_dict[file] = lmrk

lmrk_dict

{'../data/olivia/Flat feet/AM002R.obj': coord              x          y           z
 landmark                                   
 P1        162.186571  -4.535335   42.056473
 P10       -36.599817  -9.227319   73.185564
 P11        13.295744 -57.137056  110.638395
 P12        -7.685193 -57.278462   56.535825
 P2        150.406824  -6.666803   76.690649
 P3        107.048096 -11.441973    9.886203
 P4        118.469985 -20.808570   79.651264
 P5         83.673442 -13.859813   14.065540
 P6         73.610127 -42.007042   66.237575
 P7         58.485889 -47.827394   71.183925
 P8          0.623239  -4.620498   90.439360
 P9         -4.419657 -10.370667   40.643927}

In [58]:
# load meshes
import pyvista as pv

mesh_dict = {}

for file in files:
    mesh = pv.read(file)
    mesh_dict[file] = mesh

mesh_dict

{'../data/olivia/Flat feet/AM002R.obj': PolyData (0x1714b3400)
   N Cells:    557034
   N Points:   1671102
   N Strips:   0
   X Bounds:   -3.836e+01, 1.693e+02
   Y Bounds:   -1.184e+02, 8.703e-01
   Z Bounds:   9.039e+00, 1.114e+02
   N Arrays:   2}

In [59]:
# load example mesh and the extra landmarks labelling
example_mesh = pv.read(EXAMPLE_FILE)
example_lmrk = pd.read_pickle(EXAMPLE_LMRK)
example_extra_lmrk = pd.read_pickle(EXAMPLE_EXTRA_LMRK)
example_extra_lmrk

coord,x,y,z
landmark,,,
AA,-69.571717,97.848721,-14.324256
LMB,-56.088392,18.053136,-0.889815
MLA-AP,-6.568192,87.054788,-0.614215
MLA-PP,-109.917336,94.292150,-0.881860
MMB,-49.540290,60.708933,-1.221400


## Landmarks motion interpolation

In [60]:
import numpy as np
from mesh4d.analyse import measure
from scipy.interpolate import RBFInterpolator

def udmc(source_lmrk: pd.DataFrame, target_lmrk: pd.DataFrame, source_mesh: pv.core.pointset.PolyData, target_mesh: pv.core.pointset.PolyData, post_align: bool = True, post_nbr: int = 100) -> RBFInterpolator:
    # init motion field
    field = RBFInterpolator(
        source_lmrk.to_numpy(),  # (N, 3)
        target_lmrk.to_numpy(),  # (N, 3)
    )

    # post alignment
    if post_align:
        source_points = np.array(source_mesh.points)  # (N_all, 3)
        shift_points = field(source_points)
        deform_points = measure.nearest_points_from_plane(target_mesh, shift_points)
        field = RBFInterpolator(source_points, deform_points, neighbors=post_nbr)

    return field

In [61]:
from tqdm import tqdm

field_dict = {}
extra_lmrk_dict = {}

for file in tqdm(files):
    field = udmc(
        source_lmrk=example_lmrk,
        target_lmrk=lmrk_dict[file],
        source_mesh=example_mesh,
        target_mesh=mesh_dict[file],
        # post_align=False,
        post_align=True,
        )
    field_dict[file] = field
    extra_lmrk = pd.DataFrame(
        field(example_extra_lmrk.to_numpy()),
        columns=example_extra_lmrk.columns,
        index=example_extra_lmrk.index,
        )
    extra_lmrk_dict[file] = extra_lmrk
    extra_lmrk.to_pickle(file.replace(EXTENSION, '-extra-udmc.pkl'))

extra_lmrk_dict

100%|██████████| 1/1 [00:14<00:00, 14.49s/it]


{'../data/olivia/Flat feet/AM002R.obj': coord             x         y          z
 landmark                                
 AA        55.579624 -9.714337  89.834849
 LMB       58.937329 -5.151927  23.048265
 MLA-AP    99.901515 -0.210164  68.416564
 MLA-PP    20.593032  0.042928  81.367568
 MMB       67.528528 -1.599881  52.080009}

## Visual checks

In [62]:
scene = pv.Plotter()
scene.add_mesh(example_mesh, opacity=1)

scene.add_points(
    example_lmrk.to_numpy(),
    render_points_as_spheres=True,
    style='points', color='teal', point_size=20, opacity=0.9,
    )

scene.add_points(
    example_extra_lmrk.to_numpy(),
    render_points_as_spheres=True,
    style='points', color='lightcoral', point_size=20, opacity=0.9,
    )

scene.camera.roll += 160
scene.show()

Widget(value='<iframe src="http://localhost:64779/index.html?ui=P_0x1714272e0_17&reconnect=auto" class="pyvist…

In [63]:
for file in files:
    scene = pv.Plotter()
    scene.add_mesh(mesh_dict[file], opacity=1)

    scene.add_points(
        lmrk_dict[file].to_numpy(),
        render_points_as_spheres=True,
        style='points', color='teal', point_size=20, opacity=0.9,
        )

    scene.add_points(
        extra_lmrk_dict[file].to_numpy(),
        render_points_as_spheres=True,
        style='points', color='lightcoral', point_size=20, opacity=0.9,
        )

    scene.camera.roll += 160
    scene.screenshot(os.path.join(
        OUTPUT_FOLDER,
        os.path.basename(file).replace(EXTENSION, '.png'),
        ))
    scene.show()

Widget(value='<iframe src="http://localhost:64779/index.html?ui=P_0x13bcf00a0_18&reconnect=auto" class="pyvist…